In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras import layers, losses, Input
from tensorflow.keras.models import Model

2022-05-12 17:36:43.131348: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Links:

- https://blog.keras.io/building-autoencoders-in-keras.html
- https://www.tensorflow.org/tutorials/generative/autoencoder


In [2]:
pretrain_features = pd.read_csv('handout/pretrain_features.csv')
pretrain_labels = pd.read_csv('handout/pretrain_labels.csv')
train_features = pd.read_csv('handout/train_features.csv')
train_labels = pd.read_csv('handout/train_labels.csv')
test_features = pd.read_csv('handout/test_features.csv')

## Cleaning data

In [3]:
for c in pretrain_features.columns[2:]:
    if len(pretrain_features[c].unique()) < 2:
        pretrain_features = pretrain_features.drop(c, axis=1)
        train_features = train_features.drop(c, axis=1)
        test_features = test_features.drop(c, axis=1)

## Autoencoder

In [4]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            Input(shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(latent_dim),
        ])
        self.decoder = tf.keras.Sequential([
            Input(shape=(latent_dim,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.001, random_state=42
)

In [6]:
autoencoder = Autoencoder(X_pretrain_train.shape[1], latent_dim=32)
autoencoder.compile(optimizer='adam', loss=losses.MSE)

2022-05-12 17:38:08.374172: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-12 17:38:08.375872: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 17:38:08.375917: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 17:38:08.375962: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t3ui01.psi.ch): /proc/driver/nvidia/version does not exist
2022-05-12 17:38:08.376592: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild Ten

In [7]:
history = autoencoder.fit(X_pretrain_train, X_pretrain_train, epochs=15, batch_size=32, validation_data=(X_pretrain_val, X_pretrain_val))

2022-05-12 17:38:14.790034: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 389210400 exceeds 10% of free system memory.
2022-05-12 17:38:15.387400: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 389210400 exceeds 10% of free system memory.
2022-05-12 17:38:15.790100: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-12 17:38:15.790657: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294660000 Hz


Epoch 1/15
1561/1561 [==============================] - 5s 3ms/step - loss: 0.0219 - val_loss: 0.0145
Epoch 2/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0135 - val_loss: 0.0127
Epoch 3/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0118 - val_loss: 0.0116
Epoch 4/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0108 - val_loss: 0.0108
Epoch 5/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0101 - val_loss: 0.0104
Epoch 6/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0097 - val_loss: 0.0100
Epoch 7/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0094 - val_loss: 0.0098
Epoch 8/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0092 - val_loss: 0.0095
Epoch 9/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.0089 - val_loss: 0.0092
Epoch 10/15
1561/1561 [==============================] - 4s 2ms/step - loss: 0.008

## PreModel

In [8]:
class ELumon(Model):
    def __init__(self, autoencoder):
        super(ELumon, self).__init__()
        self.encoder = autoencoder.encoder
        self.regressor = tf.keras.Sequential([
            Input(shape=self.encoder.output_shape[1:]),
            layers.BatchNormalization(),
            layers.Dense(16, activation='relu'),
            layers.Dense(8, activation='relu'),
        ])
        self.top = tf.keras.Sequential([
            Input(shape=self.regressor.output_shape[1:]),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])
        # Set all layers to trainable
        for layer in self.encoder.layers:
            layer.trainable = True

    def call(self, x):
        encoded = self.encoder(x)
        reg = self.regressor(encoded)
        return self.top(reg)

In [9]:
elumon = ELumon(autoencoder)
#opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
elumon.compile(optimizer="adam", loss="mean_squared_error")

In [10]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.1, random_state=42
)

In [11]:
history_elumon = elumon.fit(X_pretrain_train, y_pretrain_train, epochs=20, batch_size=32, validation_data=(X_pretrain_val, y_pretrain_val))

Epoch 1/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.8026 - val_loss: 0.0755
Epoch 2/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0492 - val_loss: 0.0424
Epoch 3/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0295 - val_loss: 0.0270
Epoch 4/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0224 - val_loss: 0.0147
Epoch 5/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0170 - val_loss: 0.0239
Epoch 6/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0135 - val_loss: 0.0099
Epoch 7/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0118 - val_loss: 0.0106
Epoch 8/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0098 - val_loss: 0.0075
Epoch 9/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0086 - val_loss: 0.0067
Epoch 10/20
1407/1407 [==============================] - 3s 2ms/step - loss: 0.007

## Model

In [12]:
class EGap(Model):
    def __init__(self, premodel):
        super(EGap, self).__init__()
        self.encoder = premodel.encoder
        self.regressor = premodel.regressor
        self.top = tf.keras.Sequential([
            Input(shape=self.regressor.output_shape[1:]),
            layers.Dense(2, activation='relu'),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])

        for layer in self.encoder.layers:
            layer.trainable = False
        for layer in self.regressor.layers:
            layer.trainable = True
        for layer in self.top.layers:
            layer.trainable = True

    def call(self, x):
        encoded = self.encoder(x)
        reg = self.regressor(encoded)
        return self.top(reg)

In [16]:
egap = EGap(elumon)
egap.compile(optimizer="adam", loss="mean_squared_error")

In [17]:
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
    train_features[train_features.columns[2:]], train_labels[["homo_lumo_gap"]], test_size=0.02, random_state=42
)

In [18]:
egap.fit(X_train_train, y_train_train, epochs=15, validation_data=(X_train_val, y_train_val))

Epoch 1/15
4/4 [==============================] - 1s 46ms/step - loss: 7.3649 - val_loss: 3.4781
Epoch 2/15
4/4 [==============================] - 0s 10ms/step - loss: 5.8656 - val_loss: 2.6927
Epoch 3/15
4/4 [==============================] - 0s 10ms/step - loss: 4.6827 - val_loss: 2.0551
Epoch 4/15
4/4 [==============================] - 0s 17ms/step - loss: 4.0420 - val_loss: 1.5378
Epoch 5/15
4/4 [==============================] - 0s 14ms/step - loss: 2.8444 - val_loss: 1.1042
Epoch 6/15
4/4 [==============================] - 0s 12ms/step - loss: 2.0763 - val_loss: 0.7583
Epoch 7/15
4/4 [==============================] - 0s 12ms/step - loss: 1.6588 - val_loss: 0.4983
Epoch 8/15
4/4 [==============================] - 0s 13ms/step - loss: 1.3829 - val_loss: 0.3185
Epoch 9/15
4/4 [==============================] - 0s 13ms/step - loss: 1.0431 - val_loss: 0.1949
Epoch 10/15
4/4 [==============================] - 0s 12ms/step - loss: 0.8239 - val_loss: 0.1187
Epoch 11/15
4/4 [============

In [19]:
y_pred = egap.predict(test_features[test_features.columns[2:]])

In [20]:
output_df = pd.DataFrame({"Id": test_features["Id"], "y": y_pred.reshape(-1,)})
output_df.to_csv("submission.csv", index=False)